In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

### 코사인 유사도 분석

In [2]:
tableDF = pd.read_csv('./project/table.csv', index_col=0)
tableDF['image'] 

0       ./static/img/living/table/table1.png
1       ./static/img/living/table/table2.png
2       ./static/img/living/table/table3.png
3       ./static/img/living/table/table4.png
4       ./static/img/living/table/table5.png
                       ...                  
94     ./static/img/living/table/table98.png
95     ./static/img/living/table/table99.png
96    ./static/img/living/table/table100.png
97    ./static/img/living/table/table101.png
98    ./static/img/living/table/table102.png
Name: image, Length: 99, dtype: object

In [4]:
# 명사 추출
from konlpy.tag import Okt

okt = Okt()

box = []
for i in tqdm(tableDF['review']):
    a = okt.nouns(i)
    # 하나의 문장으로 합쳐지게 join 시킴
    box.append(' '.join(a))

tableDF['nouns'] = box

100%|██████████| 99/99 [00:18<00:00,  5.32it/s]


In [5]:
tableDF.head(1)

,brand,item,price,image,link,review,nouns
0,먼데이하우스,순수원목 B테이블라지 접이식 거실테이블,"37,900",./static/img/living/table/table1.png,https://ohou.se/productions/387732/selling?aff...,['이사오기 전부터 쓰던 테이블인데 치킨 박스같은거 물묻은상태로 올려놓고 방치하면 ...,이사 오기 전 테이블 치킨 박스 물 상태 방치 프린팅 상 고거 다른 얼룩 디자인 어...


In [6]:
# 중간 저장
tableDF.to_csv('tableDF.csv')

In [7]:
tableDF = pd.read_csv('./tableDF.csv', index_col=0)
tableDF.head(1)

,brand,item,price,image,link,review,nouns
0,먼데이하우스,순수원목 B테이블라지 접이식 거실테이블,"37,900",./static/img/living/table/table1.png,https://ohou.se/productions/387732/selling?aff...,['이사오기 전부터 쓰던 테이블인데 치킨 박스같은거 물묻은상태로 올려놓고 방치하면 ...,이사 오기 전 테이블 치킨 박스 물 상태 방치 프린팅 상 고거 다른 얼룩 디자인 어...


In [8]:
# 단어 카운트 기반의 벡터화
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
dtm = cv.fit_transform(tableDF['nouns']) # dtm : 텍스트를 단어의 등장 횟수로 표현된 문서-단어 행렬(document-term matrix)
dtm_table = pd.DataFrame(dtm.toarray(), columns=cv.get_feature_names_out()) # dtm.toarray() : 희소행렬인 dtm을 밀집행렬로 변환
dtm_table

,가가,가게,가격,가겯,가공,가구,가구라,가구점,가기,가까스로,...,희망,흰색,히든,히터,히히히,힐링,힐일,힘둿다,힘들엇어,힘좀
0,0,0,14,0,0,2,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,0,11,0,0,1,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0
2,0,0,13,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,5,0,0,1,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
4,0,0,10,0,0,1,0,0,0,0,...,0,6,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0,0,15,0,0,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95,0,0,14,0,0,1,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0
96,0,0,7,0,0,0,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0
97,0,0,15,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# dtm 파일 저장
dtm_table.to_csv('dtm_table.csv')

---

In [11]:
import pandas as pd 
import numpy as np
from konlpy.tag import Okt
from konlpy.tag import Kkma

tableDF = pd.read_csv('./tableDF.csv', index_col=0)
dtm_table = pd.read_csv('./dtm_table.csv', index_col=0)

# 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
# a, b : 두 벡터
def make_cosine(a,b):
    return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

kkma = Kkma()
text = '앉아도 꺼지지 않는 폭식한 소파'
strlist = kkma.nouns(text)
text = [' '.join(strlist)]

result = cv.transform(text) # 위에서 fit 시켰기 때문에 transform만 해주면 됨
input_text = result.toarray()

brand_list = np.array(tableDF['brand'])
item_list = np.array(tableDF['item'])
price_list = np.array(tableDF['price'])
image_list = np.array(tableDF['image'])

result_cosine = make_cosine(dtm_table, input_text.reshape(-1))
result_args = result_cosine.argsort()[::-1] # argsort() : 배열을 정렬했을 때의 인덱스를 반환하는 함수

# 내림차순 정렬 => [result_args][0] : 코사인 유사도가 가장 큼
print(brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0])

리바트 무니 리프트업 테이블 3colors 248,000 ./static/img/living/table/table18.png


In [12]:
# 위의 코드 함수화
def find_table(text):

    import pandas as pd 
    import numpy as np
    from konlpy.tag import Okt
    from konlpy.tag import Kkma

    tableDF = pd.read_csv('./tableDF.csv', index_col=0)
    dtm_table = pd.read_csv('./dtm_table.csv', index_col=0)

    # 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
    def make_cosine(a,b):
        return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

    kkma = Kkma()
    search_text = [text]
    strlist = kkma.nouns(search_text[0])
    search_text = [(" ".join(strlist))]

    result = cv.transform(search_text)
    search_words = result.toarray()

    brand_list = np.array(tableDF['brand'])
    item_list = np.array(tableDF['item'])
    price_list = np.array(tableDF['price'])
    image_list = np.array(tableDF['image'])

    result_cosine = make_cosine(dtm_table, search_words.reshape(-1))

    result_args = result_cosine.argsort()[::-1]
    return brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0]